<div dir="rtl" align="center">
<font face="XB Zar" size=5>
    <font face="IranNastaliq" size=5>
      به نام خدا
    </font>
    <br>
    <font size=3>
      دانشگاه صنعتی شریف - دانشکده مهندسی کامپیوتر
    </font>
    <br>
    <font color=blue size=5>
      پردازش زبان‌های طبیعی
    </font>
    <br>
    <hr/>
    <font color=red size=6>
      تمرین سوم: جستجوی دارو
    </font>
    <br>
      سینا عبدوس، مهسا یزدانی، حدیث احمدیان
    <hr>
<br>
  <div align="right">

  <div>
</div>

<div dir='rtl' style='direction:rtl:'>

# روند انجام تمرین
در ابتدا داده‌های مورد نیاز از داروها شامل نام دارو‌ها و توضیحات متنی از کاربرد آن‌ها از مجموعه داده‌ی DrugBank دانلود و استخراج می‌شود. سپس بخشی وجود دارد که به کامل کردن نام دارو به کمک auto-complete و auto-correction می‌پردازد. سپس نام نهایی مورد نظر دارو بدست آمده و از روی این نام نهایی داروهایی با کاربرد مشابه پیشنهاد داده می‌شوند.
<br>
برای پیشنهاد داروها با کاربرد مشابه ابتدا از روش ساده‌ی tf-idf استفاده می‌شود که مشاهده خواهیم کرد خروجی‌های آن مطلوب نیستند.
در ادامه به سراغ مدل‌های pre-trained می‌رویم. ابتدا مدل ‌BERT و سپیس مدل BioBERT مورد استفاده قرار می‌گیرند. مشاهده می‌کنیم که جواب این دو مدل بهتر از tf-idf است و از آن‌جایی که مدل BioBERT روی داده زیستی train شده است، از همه حالت‌ها برای پیشنهاد دادن داروهایی با کاربرد مشابه، بهتر عمل می‌کند.


<div dir='rtl' style='direction:rtl:'>

# مجموعه داده مورد استفاده
برای انجام این تمرین از مجموعه داده‌ی DrugBank استفاده شده است که مجموعه داده‌ی جامعی از دارو‌های approved و اطلاعات مختلف آن‌هاست. این اطلاعات مختلف شامل پاراگراف‌های توضیحی در مورد تاریخچه و نحوه به کارگیری دارو می‌شود که از آن‌ها بعدتر در این تمرین برای پیدا کردن داروهایی با کاربرد مشابه استفاده می‌شود.
داده‌ی DrugBank از سایت آن قابل دانلود است که به صورت یک فایل xml خواهد بود. نیاز است داده‌های مورد نیاز این تمرین از این فایل xml استخراج شود که در قسمت پیش پردازش توضیح داده می‌شود.


<div dir='rtl' style='direction:rtl:'>

# پیش پردازش ها
برای استخراج بخش هایی از داداه که به آن نیاز داشتیم یک پیش پردازش لازم بود، ما برای هر دارو اطلاعات زیر را استخراج کردیم.

* name
* DrugBank-id
* description
* indication

داده ها ابتدا از یک فایل xml جدا شده و سپس به شکل یک dataframe مرتب شده و نهایتا به فرمت یک فایل tsv ذخیره میشوند.

<font color='red' size='4'>
توجه :
</br>
<font color='blue' size='3'>
به دلیل حجم زیاد داده ها قبل از پیش پردازش (3 گیگابایت) نتیجه ی نهایی پیش پردازش  برای بخش هایی که لازم است آپلود شده  است و دانلود دیتای خام و اجرای کدهای پیش پردازش لازم نیست



In [ ]:
import pandas as pd

In [ ]:
def get_data(counter,if_count):
    data=[]
#     counter=2000
    with open('full database.xml','r',encoding='utf-8') as f:
        curr=0
        num=0
        new_drug=False
        get_drug_name=False
        get_dis=False
        get_ind=False
        get_dis2=False
        get_ind2=False
        data=[]
        ind=''
        
        
        for line in f:
            if counter<0:
                break
            if if_count:
                counter-=1
            #print(line)
            line=line.strip()
            
            
            if line.startswith("<drug type"):
                if curr!=0:
                    #print(curr)
                    data.append(drug_name)
                    data.append(drug_id)
                    data.append(dis)
                    data.append(ind)

                num+=1
                if curr<num:
                    curr=num
                    new_grug=True
                drug_name="<name></name>"
                drug_id="<drugbank-id></drugbank-id>"
                dis="<description></description>"
                ind="<indication></indication>"
                
            
            if line.startswith('<drugbank-id primary="true">'):
                get_drug_name=True
                drug_id="<drugbank-id>"+line[28:]
                
            if line.startswith('<name>') and get_drug_name:
                drug_name=line
                get_drug_name=False
                get_dis=True
                
            if line.startswith('<description>') and get_dis:
                dis=line
                get_dis=False
                get_ind=True
                if not dis.endswith('</description>'):
                    get_dis2=True
                    get_ind=False
                    
            if get_dis2:
                dis+=line
                if dis.endswith('</description>'):
                    get_dis2=False
                    get_ind=True
                
            if line.startswith('<indication>') and get_ind:
                ind=line
                get_ind=False
                if not ind.endswith('</indication>'):
                    get_ind2=True
                    
            if get_ind2:
                ind+=line
                if ind.endswith('</indication>'):
                    get_ind2=False
                
 
    return data


def transform_to_dataframe(data):
    mapping={'drugbank-id':'drugbank-id','name':'name','description':'description','indication':'indication'}
    dict_data={'drugbank-id':[],'name':[],'description':[],'indication':[]}
    for d in data:
        for column in mapping.keys():
            if d.startswith(f'<{column}>'):
                dict_data[mapping[column]].append(d[d.find(f'<{column}>')+len(column)+2:d.find(f'</{column}>')])
    return pd.DataFrame.from_dict(dict_data)

In [ ]:
counter=10000
data=get_data(counter,False)
#data
df=transform_to_dataframe(data)
df.to_csv('drugbank_drug.tsv',sep='\t')

<div dir='rtl' style='direction:rtl:'>

### پیش پردازش لازم برای جستجوی نام داروها
<font color='black'>
در این قسمت نیز نام داروهایی که استخراج شده همگی به صورت lowercase در فایلی شامل فقط نام دارو ها نوشته میشود. این فایل برای جستجوی نام دارو ها و تسک auto-compelete و auto-correction تشکیل شده است.
<br>
<font color='blue'>
نتیجه ی نهایی این بخش نیز به طور آماده دانلود میشود و نیازی به اجرای این پیش پردازش نیست.

In [ ]:
with open('drugnames_mixed', 'r') as data_file:
    with open('drugnames', 'a') as output_file:
        for line in data_file:
            output_file.write(line.lower())

<div dir='rtl' style='direction:rtl:'>
<font color='red' size='5'>
دانلود فایل های نهایی

In [1]:
# download drugnames
!gdown --id 1VRAG1gflvuRGjE-GAP4-wd5-1ZKnUpIk
# download drugbank drug data
!gdown --id 1zmYank843fJFdhgxitl-4fnW7f-uX3ec -O drugbank.tsv

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1VRAG1gflvuRGjE-GAP4-wd5-1ZKnUpIk
To: /content/drugnames
100% 432k/432k [00:00<00:00, 134MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zmYank843fJFdhgxitl-4fnW7f-uX3ec
To: /content/drugbank.tsv
100% 6.35M/6.35M [00:00<00:00, 102MB/s]


<div dir='rtl' style='direction:rtl:'>
<font size='5'>
auto-compelete and auto-correction
<font size='3'>
<br>
در این بخش یک کلاس نوشته شده است که دو تابع auto-complete و auto-correction رادارد.
برای استفاده از این توابع نام داروها از درون فایل `drugnames`خوانده شده ون سپس sort میشود تا جستجو در میان آنها برای تسک های بعدی راحت تر باشد.

<br>

## auto-complete:
این تابع برای حالتی است شخص بخش کوچکی از نام دارو را به درستی به خاطر دارد. چون نام اکثر دارو ها چند بخشی هستند، برای این قسمت زیربخش های نام تمام دارویها چندبخشی را (به اضافه ی نام کامل آنها) در دیتابیس اضافه می‌کنیم تا اگر شخص بخش های میانی دارو را به خاطر داشت نیز بتوانیم نام دارو را پیدا کنیم. این تابع سپس در میان داروها جستجو می‌کند که آیا نام وارد شده بخشی از نام یک دارو هست یا نه و اگر بود نام کامل را بازمی‌گرداند. 
<br>
نکته : این موضوع که ممکن است شخص غلط املایی در وارد کردن نام داشته باشد نیز درست است و به همین منظور تابع بعدی تعریف شده. اما مزیت این حالت (auto-complete) به حالت بعدی (auto-correction) این است که اگر شخص بخش حتی خیلی کوچکی از دارویی با نام طولانی را به درستی وارد کند، به احتمال 100% آن دارو در میان نتایج خواهد بود ولی این سناریو برای auto-correction کمی چالشی تر خواهد بود.
## auto-correction:
این تابع از کاربر بخشی (هرچند با غلط املایی) را از کاربر میگیرد و بین ورودی و نام سایر داروها jaccard similarity به ازای 2-gram ها را محاسبه می‌کند و داروهایی که بیشتر به ورودی کاربر شبیه بودند را در خروجی برمی‌گرداند.
<br>
jaccard similarity برای دو مجموعه (که در اینجا مجموعه ی 2-gram های کلمات هستند) به صورت زیر محاسبه می‌شود.

<br>

<center>

$JaccardSimilarity = \frac{A \cap B}{َA \cup B}$

$\text{ A and B are sets of words 2-grams} $


In [ ]:
! pip install textdistance
import readline
import re
import pandas as pd
import numpy as np
import textdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class Complete_Correct(object):
    def __init__(self, options1, options2):
        self.options1 = sorted(options1)
        self.options2= options2

    def complete(self, text):
        if text:
            self.matches = [s for s in self.options1 
                                if s and s.startswith(text)]
        else:
            self.matches = self.options1[:]
        return self.matches

    def correct(self,text):
      J_sim = [1-(textdistance.Jaccard(qval=2).distance(d,text)) for d in self.options2]
      return J_sim

In [ ]:
d_name1=[]
d_name2=[]
with open("drugnames") as d:
  for l in d:
    l=l[:-1]
    d_name2.append(l)
    d_name1.append(l)
    ls=l.split()
    ls=[s+": "+l for s in ls]
    d_name1+=ls[1:]
completer = Complete_Correct(d_name1,d_name2)



In [ ]:
partial_name=input("enter your input:")

enter your input:doxy


In [ ]:
readline.set_completer(completer.complete)
readline.parse_and_bind('tab: complete')

print('****************************')
print(' resutls from auto-complete ')
print('****************************')
df1 = pd.DataFrame(completer.complete(partial_name))
df1 = df1.rename(columns={0:'Drug name'})
df1

****************************
 resutls from auto-complete 
****************************


,Drug name
0,doxycycline monohydrate
1,doxylamine succinate


In [ ]:
readline.set_completer(completer.correct)
readline.parse_and_bind('tab: correct')
print("***************************")
print(" resutls from auto-correct")
print("***************************")
J_sim=completer.correct(partial_name)
df2 = pd.DataFrame(d_name2)
df2['Jaccard Similarity']=J_sim
df2 = df2.rename(columns={0:'Drug_name',1:'Jaccard Similarity'})
df2 = df2.sort_values(['Jaccard Similarity'], ascending=False)
df2=df2.reset_index(drop=True)
df2

***************************
 resutls from auto-correct
***************************


,Drug_name,Jaccard Similarity
0,didox,0.400000
1,oxygen,0.333333
2,foxy-5,0.333333
3,aurodox,0.285714
4,oxytocin,0.250000
...,...,...
15236,amisulpride,0.000000
15237,simeprevir sodium,0.000000
15238,dapagliflozin propanediol monohydrate,0.000000
15239,pegnivacogin,0.000000


In [ ]:
print("***************************")
print("setting the final drug name")
print("***************************")
k=0
while k!=1 and k!=2:
  k=int(input("the drug you were seaeching for was in :[ 1:auto-complete resutls / 2: autocorection results]"))
  if k!=1 and k!=2:
    k=int(input("value must be 1 or 2 :[ 1:auto-complete resutls / 2: autocorection results]"))
  
i=int(input("enter the df index of the chosen drug"))
if k==1:
  df=df1
else:
  df=df2
final_drug_name =df.iloc[i]['Drug_name']

***************************
setting the final drug name
***************************
the drug you were seaeching for was in :[ 1:auto-complete resutls / 2: autocorection results]2
enter the df index of the chosen drug2


<div dir='rtl' style='direction:rtl:'>

# خواندن از فایل
در این قسمت داده‌های استخراج شده از Drug‌Bank که به صورت فایل tsv ذخیره شده بودند، خواهنده شده و به فورمت DataFrame در می‌آیند. رکوردهایی که هیچ توضیح متنی ندارند، از این dataframe حذف خواهند شد.

In [2]:
import pandas as pd

In [3]:
drugbank_df_raw = pd.read_csv('drugbank.tsv', sep='\t')
drugbank_df_raw.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
drugbank_df_raw

,index,drugbank-id,name,description,indication
0,0,DB00001,Lepirudin,Lepirudin is a recombinant hirudin formed by 6...,Lepirudin is indicated for anticoagulation in ...
1,1,DB00002,Cetuximab,Cetuximab is a recombinant chimeric human/mous...,Cetuximab indicated for the treatment of local...
2,2,DB00003,Dornase alfa,Dornase alfa is a biosynthetic form of human d...,Used as adjunct therapy in the treatment of cy...
3,3,DB00004,Denileukin diftitox,A recombinant DNA-derived cytotoxic protein co...,For treatment of cutaneous T-cell lymphoma
4,4,DB00005,Etanercept,Dimeric fusion protein consisting of the extra...,Etanercept is indicated for the treatment of m...
...,...,...,...,...,...
15229,15229,DB17381,Nadofaragene firadenovec,Nadofaragene firadenovec (nadofaragene firaden...,Nadofaragene firadenovec is indicated for the ...
15230,15230,DB17382,AUM-601,AUM-601 is a highly selective pan-TRK(tropomyo...,None
15231,15231,DB17383,FN-1501,None,None
15232,15232,DB17384,Tinengotinib,None,None


In [4]:
# filter out rows with None value for both description and indication
drugbank_df = drugbank_df_raw.loc[(drugbank_df_raw['description'] != "None") | (drugbank_df_raw['indication'] != "None")]
drugbank_df.reset_index(drop=True, inplace=True)
drugbank_df

,index,drugbank-id,name,description,indication
0,0,DB00001,Lepirudin,Lepirudin is a recombinant hirudin formed by 6...,Lepirudin is indicated for anticoagulation in ...
1,1,DB00002,Cetuximab,Cetuximab is a recombinant chimeric human/mous...,Cetuximab indicated for the treatment of local...
2,2,DB00003,Dornase alfa,Dornase alfa is a biosynthetic form of human d...,Used as adjunct therapy in the treatment of cy...
3,3,DB00004,Denileukin diftitox,A recombinant DNA-derived cytotoxic protein co...,For treatment of cutaneous T-cell lymphoma
4,4,DB00005,Etanercept,Dimeric fusion protein consisting of the extra...,Etanercept is indicated for the treatment of m...
...,...,...,...,...,...
14146,15226,DB17378,MVR-T3011,MVR-T3011 is a genetically modified oncolytic ...,None
14147,15227,DB17379,(-)-Gossypol,"(-)-Gossypol or (R)-Gossypol, is the R-isomer ...",None
14148,15228,DB17380,Saposin C,"Saposin C is a lysosomal protein. Recombinant,...",None
14149,15229,DB17381,Nadofaragene firadenovec,Nadofaragene firadenovec (nadofaragene firaden...,Nadofaragene firadenovec is indicated for the ...


<div dir='rtl' style='direction:rtl:'>

# محاسبه و مقایسه امبدینگ ها بر پایه روش TF-IDF 
بر پایه این روش ابتدا کلمات کلیدی از بخش discription
هر دارو استخراج می شود. این استخراج بر پایه امتیاز خروجی این روش به ازای هر کلمه می باشد که نهایتا کلمات با امتیاز بالاتر به عنوان کلمه کلیدی معرفی می شوند.
سپس با استفاده از مدل 
Bert
و از کنار هم قرار دادن کلمات بخش قبل و میانگین گیری نهایی،
امبدینگ مرتبط با هر دارو استخراج می شود. 
در بخش پایانی کار نیز از فاصله کسینوسی برای بررسی میزان شباهت امبدینگ های استخراجی استفاده می شود.
عملکرد ضعیف این روش مشهود است

In [ ]:
!pip install simpletransformers -q
!pip install scipy

<div dir='rtl' style='direction:rtl:'>

## انجام پیش پردازش

In [ ]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text
 
drugbank_df_preprocessed = pd.DataFrame()
drugbank_df_preprocessed['name'] = drugbank_df['name']
drugbank_df_preprocessed['description'] = drugbank_df['description'].apply(lambda x:pre_process(str(x)))

#show the first 'text'
# drugbank_df_preprocessed['description'][2]

# Assuming you have a DataFrame called 'df' with columns 'Name' and 'Description'
# Replace 'df' with the actual name of your DataFrame

# Find the row where 'Name' column matches 'final_dug_name'
matching_row = drugbank_df_preprocessed[drugbank_df_preprocessed['name'] == final_drug_name]

# Get the description value from the matching row
matching_row = matching_row['description']

print(str(matching_row))


'dornase alfa is a biosynthetic form of human deoxyribunuclease i dnase i enzyme it is produced in genetically modified chinese hamster ovary cho cells using recombinant dna technology the amino acid sequence of dornase alfa is identical to the endogenous human enzyme dornase alfa cleaves extracellular dna to phosphodinucleotide and phosphooligonucleotide end products without affecting intracellular dna in individuals with cystic fibrosis extracellular dna which is an extremely viscous anion is released by degenerating leukocytes that accumulate during inflammatory responses to infections enzymatic breakdown of this extracellular dna appears to reduce sputum viscosity and viscoelasticity '

In [ ]:
drugbank_df_preprocessed

Downloading...
From: https://drive.google.com/uc?id=1zL2eXR2bcqHEh8iltH2sKiD0xhm7bbUK
To: /content/stopwords.txt
100% 5.31k/5.31k [00:00<00:00, 17.3MB/s]


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  warnings.warn(


(7759, 12927)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(stop_set)

#load a set of stop words
!gdown 1zL2eXR2bcqHEh8iltH2sKiD0xhm7bbUK
stopwords=get_stop_words("/content/stopwords.txt")

#get the text column 
docs=drugbank_df_preprocessed['description'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

In [ ]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape
list(cv.vocabulary_.keys())[:10]

['lepirudin',
 'identical',
 'natural',
 'substitution',
 'leucine',
 'isoleucine',
 'terminal',
 'end',
 'molecule',
 'absence']

<div dir='rtl' style='direction:rtl:'>

## محاسبه IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [ ]:
tfidf_transformer.idf_

array([9.26359043, 8.57044325, 9.26359043, ..., 9.26359043, 9.26359043,
       9.26359043])

<div dir='rtl' style='direction:rtl:'>

## محاسبه TF-IDF

In [ ]:
# read test docs into a dataframe and concatenate title and body

drugbank_df_preprocessed['description'] =drugbank_df_preprocessed['description'] .apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=drugbank_df_preprocessed['description'].tolist()

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
# you only needs to do this once
feature_names=cv.get_feature_names_out()

# get the document that we want to extract keywords from
doc=docs_test[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Text=====")
print(docs_test[0])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Text=====
lepirudin is identical to natural hirudin except for substitution of leucine for isoleucine at the n terminal end of the molecule and the absence of a sulfate group on the tyrosine at position it is produced via yeast cells bayer ceased the production of lepirudin refludan effective may 

===Keywords===
lepirudin 0.515
refludan 0.258
isoleucine 0.246
leucine 0.232
bayer 0.219
substitution 0.21
absence 0.208
yeast 0.2
end 0.198
sulfate 0.194


In [ ]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Text=====")
    print(docs_test[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])


keywords_score = []
# for idx in range(len(docs_test)): # colab run out of memory
for idx in range(500):
  keywords_score.append([get_keywords(idx)])

# keywords_score

<div dir='rtl' style='direction:rtl:'>

## محاسبه امبدینگ هر دارو به کمک میانگین گیری از امبدینگ هر کلمه کلیدی

In [ ]:
sentences = []
sentence = ""
for item in keywords_score:
  for keyword in item[0].keys():
    sentence = sentence + " " + keyword
  
  sentences.append([sentence])
sentences

In [ ]:
from transformers.utils import SENTENCEPIECE_UNDERLINE
from simpletransformers.language_representation import RepresentationModel
        
# sentences = ["Machine Learning and Deep Learning are part of AI", "Data Science will excel in future"] #it should always be a list

sentences1 = []
sentences2 = []

model = RepresentationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=False
    )

word_vectors = []
for sentence in sentences:
  word_vectors.append(model.encode_sentences(sentence, combine_strategy=None))



In [ ]:
average_embedding = []
for word_vector in word_vectors:
  average_embedding.append(np.mean(word_vector, dtype=np.float64))

In [ ]:
print(average_embedding) 

[-0.011142750909364452, -0.011243309384493376, -0.011810755694214158, -0.012114528004363793, -0.011945770104489554, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.012302779299894553, -0.0123027

In [ ]:
import scipy 

query = 0
query_embedding = average_embedding[query]

def calculate_cosine_distance(a, b):
    cosine_distance = float(scipy.spatial.distance.cosine(a, b))
    return cosine_distance

cosine_distances = []
for index, item in enumerate(average_embedding):
  cosine_distance = calculate_cosine_distance(np.array([query_embedding]), np.array([item]))
  cosine_distances.append([cosine_distance, index])

In [ ]:
cosine_distances

In [ ]:
Index = 0
Array_sort = np.array(cosine_distances)[np.array(cosine_distances)[:,Index].argsort()]

<div dir='rtl' style='direction:rtl:'>

## 10 داروی مشابه

In [ ]:
indexes = []
for index, i in enumerate(Array_sort):
  indexes.append(int(i[1]))
  if index == 10:
    break

In [ ]:
print("query drug is : ",drugbank_df.iloc[0]['name'])
for index in indexes:
  print("suggested drug {} is : ".format(index) ,drugbank_df.iloc[index]['name'])

query drug is :  Lepirudin
suggested drug 0 is :  Lepirudin
suggested drug 341 is :  Minoxidil
suggested drug 340 is :  Clobazam
suggested drug 339 is :  Nitisinone
suggested drug 338 is :  Trimethadione
suggested drug 337 is :  Alfuzosin
suggested drug 336 is :  Aminohippurate
suggested drug 335 is :  Protriptyline
suggested drug 334 is :  Diltiazem
suggested drug 333 is :  Terfenadine
suggested drug 332 is :  Cetirizine


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<div dir='rtl' style='direction:rtl:'>

# استفاده از BERT و Bio‌BERT
در این قسمت ابتدا جملات موجود در مجموعه داده استخراج می‌شوند. سپس مدل‌های BERT و BioBERT روی این مجموعه جملات استفاده شده و به ازای هر دارو، با توجه به توضیحات آن، embeddingای بدست می‌آید.

In [5]:
# preparing sentences

sentences = list()
for index, row in drugbank_df.iterrows():
  sentence = row['indication']
  if sentence == 'None':
    sentence = row['description']
  sentences.append(sentence)

sentences[:3]

['Lepirudin is indicated for anticoagulation in adult patients with acute coronary syndromes (ACS) such as unstable angina and acute myocardial infarction without ST elevation. In patients with ACS, lepirudin is intended for use with [aspirin].[L41539] Lepirudin is also indicated for anticoagulation in patients with heparin-induced thrombocytopenia (HIT) and associated thromboembolic disease in order to prevent further thromboembolic complications.[L41539]',
 'Cetuximab indicated for the treatment of locally or regionally advanced squamous cell carcinoma of the head and neck in combination with radiation therapy. It is indicated for treating a recurrent locoregional disease or metastatic squamous cell carcinoma of the head and neck in combination with platinum-based therapy with fluorouracil. It is indicated for recurrent or metastatic squamous cell carcinoma of the head and neck progressing after platinum-based therapy.[L30448]&#13;<indication>Cetuximab indicated for the treatment of 

In [6]:
!pip install simpletransformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 112.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

<div dir='rtl' style='direction:rtl:'>

### تعریف مدل BERT

In [7]:
from simpletransformers.language_representation import RepresentationModel

model = RepresentationModel(
        model_type="bert",
        model_name="bert-base-uncased",
        use_cuda=True
    )
bert_sentence_vectors = model.encode_sentences(sentences, combine_strategy="mean")
print(bert_sentence_vectors.shape)
# print(sentence_vectors)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(14151, 768)


In [ ]:
import numpy as np
np.save('bert', bert_sentence_vectors)

In [8]:
!pip install -U sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


<div dir='rtl' style='direction:rtl:'>

### تعریف مدل BioBERT

In [9]:
from sentence_transformers import SentenceTransformer

# pain killer: mefenamic acid, ibuprofen, indomethacin, naproxen, celecoxib
# antihistamine: diphenhydramine, dimenhydrinate, hydroxyzine, cetirizine, ketotifen
# zed eneghad: heparin, enoxaparin,,, Warfarin, rivaroxaban
# anti butic: cefalexin, cefixime, ceftriaxone, cefotaxime, ceftazidime, cefazolin
# shol konande azole: methocarbamol, baclofen
# khab avar: diazepam, oksazepam, lorazepam, alprazolam, clonazepam 

# sentences = ["Acetaminophen is an analgesic drug used alone or in combination with opioids for pain management, and as an antipyretic agent.",
#              "Mefenamic acid is an NSAID used to treat mild to moderate pain for no more than a week, and primary dysmenorrhea",
#              "Cefixime is a third generation cephalosporin used to treat susceptible Gram negative and Gram positive bacterial infections.",
#              "Lepirudin is a protein-based direct thrombin inhibitor used as an anticoagulant in patients with heparin-induced thrombocytopenia.",
#              "Ibuprofen is an NSAID and non-selective COX inhibitor used to treat mild-moderate pain, fever, and inflammation.",
#              "Dimenhydrinate is a medication used to prevent and treat nausea, vomiting, vertigo, and motion sickness.",
#              "Warfarin is a vitamin K antagonist used to treat venous thromboembolism, pulmonary embolism, thromboembolism with atrial fibrillation, thromboembolism with cardiac valve replacement, and thromboembolic events post myocardial infarction.",
#              "Rivaroxaban is a factor Xa inhibitor used to treat deep vein thrombosis (DVT) and pulmonary embolism (PE). May also be used as thrombosis prophylaxis in specific situations.",
#              "Cephalexin is a first generation cephalosporin used to treat certain susceptible bacterial infections.",
#              "Diphenhydramine is a H1 receptor antihistamine used in the treatment of seasonal allergies, and various allergic reactions including sneezing, runny nose, itchy/watery eyes, itching of nose or throat, pruritus, urticaria, insect bites/stings, and allergic rashes."] #it should always be a list

model = SentenceTransformer('pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb')
biobert_sentence_vectors = model.encode(sentences)
print(biobert_sentence_vectors.shape)


(14151, 768)


In [ ]:
np.save('biobert', biobert_sentence_vectors)

<div dir='rtl' style='direction:rtl:'>

## تعریف معیار برای سنجش فاصله
در این قسمت برای پیدا کردن داروهایی با کاربرد مشابه در فضای embedding، نیاز به معیاری برای اندازه گیری فاصله داریم. در سلول بعدی تعدادی از این معیارها تعریف شده‌اند که ما در نهایت از معیار cosine similarity استفاده می‌کنیم و به کمک آن، به ازای هر نام دارو ۵ داروی مشابه به آن را گزارش می‌کنیم.

In [10]:
import numpy as np

def cosine_similarity(query_embedding, embedding_list):
    query_norm = np.linalg.norm(query_embedding)
    embedding_list_norm = np.linalg.norm(embedding_list, axis=1)
    dot_products = np.dot(embedding_list, query_embedding)
    similarities = dot_products / (embedding_list_norm * query_norm)
    return similarities


def euclidean_distance(query_embedding, embedding_list):
    distances = np.linalg.norm(embedding_list - query_embedding, axis=1)
    return distances


def manhattan_distance(query_embedding, embedding_list):
    distances = np.sum(np.abs(embedding_list - query_embedding), axis=1)
    return distances


def minkowski_distance(query_embedding, embedding_list, p):
    distances = np.power(np.sum(np.power(np.abs(embedding_list - query_embedding), p), axis=1), 1/p)
    return distances


def correlation_distance(query_embedding, embedding_list):
    correlations = 1 - np.corrcoef(embedding_list, query_embedding)[0, 1:]
    return correlations




def top_k_similar(query_embedding, embedding_list, k):
    similarities = cosine_similarity(query_embedding, embedding_list)
    indices = np.argsort(similarities)[::-1][:k]
    return indices


In [16]:
# This part is for test
final_drug_name = "Acetaminophen"
# final_drug_id = "DB01050"

drug_df = drugbank_df_raw.loc[drugbank_df_raw['name'] == final_drug_name]
drug_df

,index,drugbank-id,name,description,indication
305,305,DB00316,Acetaminophen,"Acetaminophen (paracetamol), also commonly kno...","In general, acetaminophen is used for the trea..."


<div dir='rtl' style='direction:rtl:'>

### خروجی
در این قسمت داروهای پیشنهادی مدل BERT و مدل BioBERT نمایش داده شده‌اند.

In [17]:
def find_indices(drug_idx, sentence_vectors, k=3): # k is number of top embeddings to retrieve
  query_embedding = sentence_vectors[drug_idx]
  embedding_list = sentence_vectors

  top_k_indices = top_k_similar(query_embedding, embedding_list, k)
  return top_k_indices

drug_df = drugbank_df.loc[drugbank_df['name'] == final_drug_name]

if drug_df.empty:
  print('This drug has no description or indication to in our dataset so we can not find similar drugs with it.')
else:
  query_drug_idx = drug_df.iloc[0, drug_df.columns.get_loc('index')]
  k = 6

  bert_top_indices = find_indices(query_drug_idx, bert_sentence_vectors, k)
  bert_proposed_drugs = drugbank_df.loc[bert_top_indices]

  biobert_top_indices = find_indices(query_drug_idx, biobert_sentence_vectors, k)
  biobert_proposed_drugs = drugbank_df.loc[biobert_top_indices]


In [18]:
bert_proposed_drugs[1:]

,index,drugbank-id,name,description,indication
307,307,DBSALT000031,Codeine sulfate anhydrous,Codeine Phosphate 30 mg/ml,Codeine sulfate is a form of this drug that is...
1478,1480,DBSALT003135,Dihydrocodeine thiocyanate,Dihydrocodeine may increase the central nervou...,Dihydrocodeine is used for the treatment of mo...
316,316,DBSALT002251,Hydromorphone sulfate,Dilaudid 2 mg tablet,Hydromorphone is indicated for the management ...
420,420,DBSALT003068,Prochlorperazine mesilate,Prochlorperazine 5 mg/ml vial,Indicated for the symptomatic treatment of sev...
560,560,DBSALT000538,Clonidine hydrochloride,Catapres 0.1 mg tablet,Clonidine tablets and transdermal systems are ...


In [19]:
biobert_proposed_drugs[1:]

,index,drugbank-id,name,description,indication
9197,9279,DBSALT002682,Phenyltoloxamine hydrochloride,cytosol,The primary therapeutic use for which phenylto...
938,938,DBSALT001602,Hydrocodone polistirex,Hydrocodone-Acetaminophen 10-750 mg tablet,Hydrocodone is indicated for the management of...
307,307,DBSALT000031,Codeine sulfate anhydrous,Codeine Phosphate 30 mg/ml,Codeine sulfate is a form of this drug that is...
1356,1356,DB01419,Antrafenine,Antrafenine is a piperazine derivative drug th...,Antrafenine is used as an anti-inflammatory an...
410,410,DB00423,Methocarbamol,Methocarbamol was developed in the early 1950s...,Methocarbamol tablets and intramuscular inject...


In [ ]:
from google.colab import drive
drive.flush_and_unmount()